In [18]:
import os
import pandas as pd
import numpy as np
import requests

from data_container import load_scouted_data


# CONFIG

TBA_KEY = os.getenv("TBA_KEY")  # set in your environment
EVENT_KEY = "2026paca" #gpr code

if not TBA_KEY:
    print("⚠️ No TBA_KEY set. Running in offline/pre-event mode (no TBA merge).")


# load scouting data

df = load_scouted_data().copy()

TEAM_COL  = "Team Number" if "Team Number" in df.columns else ("Team" if "Team" in df.columns else None)
MATCH_COL = "Match Number" if "Match Number" in df.columns else ("Match" if "Match Number" in df.columns else None)

if TEAM_COL is None or MATCH_COL is None:
    raise ValueError(f"Missing required cols. Have columns: {df.columns.tolist()}")

for c in ["Auto Fuel", "Teleop Fuel"]:
    df[c] = pd.to_numeric(df.get(c, 0), errors="coerce").fillna(0)

df["Total Fuel in Hub"] = df["Auto Fuel"] + df["Teleop Fuel"]


# pulling matches from TBA (although matches have not started)

matches = []
if TBA_KEY:
    headers = {"X-TBA-Auth-Key": TBA_KEY}
    url = f"https://www.thebluealliance.com/api/v3/event/{EVENT_KEY}/matches/simple"
    resp = requests.get(url, headers=headers)

    print("TBA STATUS:", resp.status_code)
    if resp.status_code == 200:
        matches = resp.json()
    else:
        print("TBA BODY (first 200):", resp.text[:200])
        print("⚠️ TBA request failed; continuing without TBA merge.")


# build lookup if matches exist (will work once they do)

tba_lookup = pd.DataFrame(columns=["Match ID","Comp Level","Match Number","Team Number","Alliance"])

if isinstance(matches, list) and len(matches) > 0:
    lookup_rows = []
    for m in matches:
        comp = m["comp_level"]
        match_num = m["match_number"]
        set_num = m.get("set_number", 1)

        match_id = f"{comp.upper()}{match_num}"
        if comp != "qm":
            match_id = f"{comp.upper()}-{set_num}-{match_num}"

        for t in m["alliances"]["red"]["team_keys"]:
            lookup_rows.append({
                "Match ID": match_id,
                "Comp Level": comp,
                "Match Number": match_num,
                "Team Number": int(t.replace("frc", "")),
                "Alliance": "Red"
            })
        for t in m["alliances"]["blue"]["team_keys"]:
            lookup_rows.append({
                "Match ID": match_id,
                "Comp Level": comp,
                "Match Number": match_num,
                "Team Number": int(t.replace("frc", "")),
                "Alliance": "Blue"
            })

    tba_lookup = pd.DataFrame(lookup_rows)


# merge if lookup exists; else create placeholder cols so that i can commit

if not tba_lookup.empty:
    df = df.merge(
        tba_lookup,
        left_on=[MATCH_COL, TEAM_COL],
        right_on=["Match Number", "Team Number"],
        how="left"
    )
else:
    # pre-event mode: create columns so code doesn't crash
    df["Match ID"] = pd.NA
    df["Alliance"] = pd.NA
    df["Comp Level"] = pd.NA

ALLI_COL = "Alliance"


# avg alliance fuel stats (empty till szn)

if df["Match ID"].notna().any() and df[ALLI_COL].notna().any():
    alliance_stats = (
        df.groupby(["Match ID", ALLI_COL])
          .agg(
              alliance_total_fuel=("Total Fuel in Hub", "sum"),
              alliance_avg_fuel=("Total Fuel in Hub", "mean"),
              teams_count=(TEAM_COL, "nunique"),
          )
          .reset_index()
          .sort_values(["Match ID", ALLI_COL])
    )
else:
    alliance_stats = pd.DataFrame(columns=[
        "Match ID", "Alliance", "alliance_total_fuel", "alliance_avg_fuel", "teams_count"
    ])


# prediction r/b (empty till szn)

if not alliance_stats.empty:
    pivot = alliance_stats.pivot(index="Match ID", columns=ALLI_COL, values="alliance_total_fuel").reset_index()
    if "Red" not in pivot.columns:  pivot["Red"] = 0
    if "Blue" not in pivot.columns: pivot["Blue"] = 0
    pivot["Fuel_Diff"] = pivot["Red"] - pivot["Blue"]
    pivot["Predicted_Winner"] = np.where(pivot["Fuel_Diff"] >= 0, "Red", "Blue")
else:
    pivot = pd.DataFrame(columns=["Match ID","Red","Blue","Fuel_Diff","Predicted_Winner"])


# exports to csv

radio_df = pivot[["Match ID","Red","Blue","Fuel_Diff","Predicted_Winner"]] if not pivot.empty else pivot
radio_df.to_csv("match_predictions_all_matches.csv", index=False)
alliance_stats.to_csv("alliance_stats.csv", index=False)
tba_lookup.to_csv("tba_lookup.csv", index=False)

print("✅ Exported CSVs:")
print(" - match_predictions_all_matches.csv")
print(" - alliance_stats.csv")
print(" - tba_lookup.csv")
if tba_lookup.empty:
    print("⚠️ Note: TBA returned no matches yet (pre-event). Exports are empty placeholders.")

⚠️ No TBA_KEY set. Running in offline/pre-event mode (no TBA merge).
✅ Exported CSVs:
 - match_predictions_all_matches.csv
 - alliance_stats.csv
 - tba_lookup.csv
⚠️ Note: TBA returned no matches yet (pre-event). Exports are empty placeholders.
